# __Notes on Object-Oriented Programming and the Binomial Model__

In [1]:
import numpy as np
from scipy.stats import binom

The objective in this exercise is to create types that can be used in Python that represent objects in the real world that we are interested in. 

<br>

We start by creating a `VanillaCallOption` type that will have two data members (`strike` and `expiry`) and one form of behavior (the `payoff` function):

In [2]:
class VanillaCallOption:
    def __init__(self, strike, expiry):
        self.strike = strike
        self.expiry = expiry
    
    def payoff(self, spot):
        return np.maximum(spot - self.strike, 0.0)

If we want to represent a put option we can copy/paste and modify as follows:

In [3]:
class VanillaPutOption:
    def __init__(self, strike, expiry):
        self.strike = strike
        self.expiry = expiry
    
    def payoff(self, spot):
        return np.maximum(self.strike - spot, 0.0)

We can now instantiate objects of these types as follows:

In [4]:
the_call = VanillaCallOption(40.0, 1.0)
the_put = VanillaPutOption(40.0, 1.0)

In [5]:
the_call.payoff(41.0)

1.0

In [6]:
the_put.payoff(39.0)

1.0

We can can create objects in memory that model the real world objects that we are interested in. 

<br>

But, our example has some pretty serious code smells (https://en.wikipedia.org/wiki/Code_smell). Any time you copy and paste code you can know what your code is pretty smelly. 

<br>

It would be much better to have a single class type that is composable, i.e. could take on either call or put payoff behavior at run time. 

<br>

See here for information about composability: https://en.wikipedia.org/wiki/Composability

<br>

The basic idea is that if you are a user of my module you could create new types of behavior from the types that I provide you without requiring that my code be re-written. Also, I could update algorithms in my module and ship them and once you import them you could benefit from improvements in the algorithm without re-writing your code. If we can achieve this it is a very powerful outcome for our domain (which we all know is ARBITRAGE!)

<br>

Let's improve the code by making it more composable. But first, let's improve the code by making the encapsulation stronger. 

In [54]:
class VanillaOption:
    def __init__(self, strike, expiry):
        self.__strike = strike
        self.__expiry = expiry
        
    @property
    def strike(self):
        return self.__strike
    
    @strike.setter
    def strike(self, new_strike):
        self.__strike = new_strike
        
    @property
    def expiry(self):
        return self.__expiry
    
    @expiry.setter
    def expiry(self, new_expiry):
        self.__expiry = new_expiry
    
    def payoff(self, spot):
        return np.maximum(spot - self.__strike, 0.0)

In [55]:
the_call = VanillaOption(40.0, 1.0)
the_call.strike 

40.0

In [56]:
the_call.payoff(41.0)

1.0

In [57]:
the_call.strike = 39.0
the_call.strike

39.0

In [16]:
the_call.payoff(41.0)

2.0

We now have a public interface to allow obtaining and setting the values of the data members that honors the principle of encapsulation. 

Let's now turn to making our type composable.

<br>
    
We start by recognizing that functions in Python are first-class objects, which means we can pass them around as other types of data. 

<br>

Let's take a detour with a simpler example. 

<br>

In [17]:
class Math:
    def __init__(self, x, y, operation):
        self.x = x
        self.y = y
        self.__operation = operation
        
    def operation(self):
        return self.__operation(self.x, self.y)

This simple class will perform basic mathematical operations. It will have two data members representing floating point values. The third argument to the constructor function is a function itself. This will allow us to swap out at run time the actual behavior of the method. 

<br>

We will create two functions, one for addition and one for subtraction.

<br>

In [22]:
def addition(a, b):
    return a + b

In [23]:
def subtraction(a, b):
    return a - b

We can now use these to define an object with behavior.

In [28]:
obj1 = Math(2, 1, addition)

In [29]:
obj1.operation()

3

In [30]:
obj2 = Math(2,1, subtraction)
obj2.operation()

1

Let's now imagine that I ship this code to you and you would like to create objects that have behavior that you would like to define. So you create the following two functions (for multiplication and division).

In [31]:
def multiplication(a, b):
    return a * b

In [32]:
def division(a, b):
    return a / b

In [35]:
obj3 = Math(6, 3, multiplication)

In [36]:
obj3.operation()

18

In [37]:
obj4 = Math(6, 3, division)
obj4.operation()

2.0

We can see that our code is fully composable. You were able to add additional behavior without changing a single liine of the class definition. This is very powerful. 

<br>

We seek objects that we can instantiate in the language of our domain (ARBITRAGE) that give us mental constructs to perform our tasks. 

<br>

We can now refine our class as follows. 

In [39]:
def call_payoff(option, spot):
    return np.maximum(spot - option.strike, 0.0)

In [40]:
def put_payoff(option, spot):
    return np.maximum(option.strike - spot, 0.0)

In [45]:
class VanillaOption:
    def __init__(self, strike, expiry, payoff):
        self.__strike = strike
        self.__expiry = expiry
        self.__payoff = payoff
        
    @property
    def strike(self):
        return self.__strike
    
    @strike.setter
    def strike(self, new_strike):
        self.__strike = new_strike
        
    @property
    def expiry(self):
        return self.__expiry
    
    @expiry.setter
    def expiry(self, new_expiry):
        self.__expiry = new_expiry
    
    def payoff(self, spot):
        return self.__payoff(self, spot)

In [46]:
spot = 41.0
the_call = VanillaOption(40.0, 1.0, call_payoff)
the_call.payoff(spot)

1.0

In [47]:
spot = 39.0
the_put = VanillaOption(40.0, 1.0, put_payoff)
the_put.payoff(spot)

1.0

Notice that our class is composable in exactly the same way as our simple `Math` class above. I could ship this code to you and you could add your own payoffs and the class would obtain new behavior without changing the class definition. We will do this later for new payoff types. 

## The European Binomial Model

Let's now take a look at the European Binomial Model algorithm. One thing we will notice is that the algorithm is fully _polymorphic_, meaning that without changing a single line of code the algorithm with price either a call or a put option.

In [48]:
def european_binomial(option, spot, rate, vol, div, steps):
    strike = option.strike
    expiry = option.expiry
    call_t = 0.0
    spot_t = 0.0
    h = expiry / steps
    num_nodes = steps + 1
    u = np.exp((rate - div) * h + vol * np.sqrt(h))
    d = np.exp((rate - div) * h - vol * np.sqrt(h))
    pstar = (np.exp(rate * h) - d) / ( u - d)
    
    for i in range(num_nodes):
        spot_t = spot * (u ** (steps - i)) * (d ** (i))
        call_t += option.payoff(spot_t) * binom.pmf(steps - i, steps, pstar)

    call_t *= np.exp(-rate * expiry)
    
    return call_t

In [51]:
call_prc = european_binomial(the_call, 41.0, 0.08, 0.3, 0.0, 100)
print(f"The Call Option Price is: {call_prc : 0.2f}")

The Call Option Price is:  6.97


In [53]:
put_prc = european_binomial(the_put, 41.0, 0.08, 0.3, 0.0, 100)
print(f"The Put Option Price is: {put_prc : 0.2f}")

The Put Option Price is:  2.89


The objective is arbitrage. We are well on our way. 